In [1]:
import numpy as np
import pandas as pd
import operator
import pickle

In [2]:
## Reading MIMIC Data

data_dir = "../Data/"
drug_codes = pd.read_csv(data_dir+"DRGCODES.csv.gz", compression="gzip")
icu_stays = pd.read_csv(data_dir+"ICUSTAYS.csv.gz", compression="gzip")
#input_events_cv = pd.read_csv(data_dir+"INPUTEVENTS_CV.csv.gz", compression="gzip")
#input_events_mv = pd.read_csv(data_dir+"INPUTEVENTS_MV.csv.gz", compression="gzip")
#lab_events = pd.read_csv(data_dir+"LABEVENTS.csv.gz", compression="gzip")
microbiology_events = pd.read_csv(data_dir+"MICROBIOLOGYEVENTS.csv.gz", compression="gzip")
#note_events = pd.read_csv(data_dir+"NOTEEVENTS.csv.gz", compression="gzip")

In [3]:
diagnoses_icd = pd.read_csv(data_dir+"DIAGNOSES_ICD.csv.gz", compression="gzip")
d_icd_diagnoses = pd.read_csv(data_dir+"D_ICD_DIAGNOSES.csv.gz", compression="gzip")

In [5]:
d_icd_diagnoses.head()

ROW_ID ICD9_CODE               SHORT_TITLE  \
0     174     01166     TB pneumonia-oth test   
1     175     01170    TB pneumothorax-unspec   
2     176     01171   TB pneumothorax-no exam   
3     177     01172  TB pneumothorx-exam unkn   
4     178     01173  TB pneumothorax-micro dx   

                                          LONG_TITLE  
0  Tuberculous pneumonia [any form], tubercle bac...  
1              Tuberculous pneumothorax, unspecified  
2  Tuberculous pneumothorax, bacteriological or h...  
3  Tuberculous pneumothorax, bacteriological or h...  
4  Tuberculous pneumothorax, tubercle bacilli fou...

In [4]:
#output_events = pd.read_csv(data_dir+"OUTPUTEVENTS.csv.gz", compression="gzip")
patients = pd.read_csv(data_dir+"PATIENTS.csv.gz", compression="gzip")
#prescriptions = pd.read_csv(data_dir+"PRESCRIPTIONS.csv.gz", compression="gzip")
procedure_events_mv = pd.read_csv(data_dir+"PROCEDUREEVENTS_MV.csv.gz", compression="gzip")
procedures_icd = pd.read_csv(data_dir+"PROCEDURES_ICD.csv.gz", compression="gzip")
services = pd.read_csv(data_dir+"SERVICES.csv.gz", compression="gzip")
transfers = pd.read_csv(data_dir+"TRANSFERS.csv.gz", compression="gzip")

In [5]:
prescriptions = pd.read_csv(data_dir+"PRESCRIPTIONS.csv.gz", compression="gzip")
prescriptions.head(5)

/share/apps/python3/3.5.3/intel/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


ROW_ID  SUBJECT_ID  HADM_ID  ICUSTAY_ID            STARTDATE  \
0  2214776           6   107064         NaN  2175-06-11 00:00:00   
1  2214775           6   107064         NaN  2175-06-11 00:00:00   
2  2215524           6   107064         NaN  2175-06-11 00:00:00   
3  2216265           6   107064         NaN  2175-06-11 00:00:00   
4  2214773           6   107064         NaN  2175-06-11 00:00:00   

               ENDDATE DRUG_TYPE            DRUG DRUG_NAME_POE  \
0  2175-06-12 00:00:00      MAIN      Tacrolimus    Tacrolimus   
1  2175-06-12 00:00:00      MAIN        Warfarin      Warfarin   
2  2175-06-12 00:00:00      MAIN  Heparin Sodium           NaN   
3  2175-06-12 00:00:00      BASE             D5W           NaN   
4  2175-06-12 00:00:00      MAIN      Furosemide    Furosemide   

  DRUG_NAME_GENERIC FORMULARY_DRUG_CD     GSN          NDC  \
0        Tacrolimus             TACR1  021796  469061711.0   
1          Warfarin             WARF5  006562   56017275.0   
2               NaN        HEPAPREMIX  006522  338055002.0   
3               NaN           HEPBASE     NaN          0.0   
4        Furosemide            FURO20  008208   54829725.0   

            PROD_STRENGTH DOSE_VAL_RX DOSE_UNIT_RX FORM_VAL_DISP  \
0             1mg Capsule           2           mg             2   
1              5mg Tablet           5           mg             1   
2  25,000 unit Premix Bag      25,000         UNIT             1   
3            HEPARIN BASE         250           ml           250   
4             20mg Tablet          20           mg             1   

  FORM_UNIT_DISP ROUTE  
0            CAP    PO  
1            TAB    PO  
2            BAG    IV  
3             ml    IV  
4            TAB    PO

ICD9 Codes for Depression
Code: 311     No. of subjects: 2926       No. of morphine prescribed subjects
311           2926                        4
29690         69                          0
29699         4                           0
3000          0                           0

In [21]:
opioids_effects_list = ["30400", "30401", "30402", "30403", "30470", "30471", "30472", 
                               "30473", "30550", "30551", "30552", "30553", "96500", "96501", "96502", "96509"]
psychological_effects_list = ["30410", "30411", "30412", "30413", "30540", "30541", "30542", 
                              "30543"]
cocaine_dependence_list = ["30420", "30421", "30422", "30423"]
cannabis_dependence_list = ["30430", "30431", "30432", "30433"]
amphetamine_psychostimulant_list = ["30440", "30441", "30442", "30443"]
hallucinogen_list = ["30450", "30451", "30452", "30453"]
poisoning_list = ["96502", "96509", "9701", "E8500", "E8501", "E8502"]
total_effects_list = opioids_effects_list + psychological_effects_list + cocaine_dependence_list + cannabis_dependence_list + amphetamine_psychostimulant_list + hallucinogen_list + poisoning_list

In [7]:
with open('../Data/Opioids/Subjects/narcotic_subject_df.pickle', 'rb') as pickler:
    narcotic_subject_df = pickle.load(pickler)
with open('../Data/Opioids/Subjects/anti_narcotic_subject_df.pickle', 'rb') as pickler:
    anti_narcotic_subject_df = pickle.load(pickler)

In [15]:
subjects_with_narcotic_drugs = set(narcotic_subject_df["SUBJECT_ID"])
subjects_with_anti_narcotic_drugs = set(anti_narcotic_subject_df["SUBJECT_ID"])
print("Narcotic Subjects: %i" %(len(subjects_with_narcotic_drugs)))
print("Anti Narcotic Subjects: %i" %(len(subjects_with_anti_narcotic_drugs)))

Narcotic Subjects: 29959
Anti Narcotic Subjects: 1437


In [9]:
subjects_with_narcotic_and_anti_narcotic = subjects_with_narcotic_drugs.union(subjects_with_anti_narcotic_drugs)
print("Narcotic and Anti Narcotic Subjects Unioin: %i" %(len(subjects_with_narcotic_and_anti_narcotic)))

Narcotic and Anti Narcotic Subjects Unioin: 29991


In [10]:
print("")

In [26]:
diagnoses_with_effects_df = diagnoses_icd[diagnoses_icd["ICD9_CODE"].isin(total_effects_list)]
subjects_with_effects = set(diagnoses_icd[diagnoses_icd["ICD9_CODE"].isin(total_effects_list)]["SUBJECT_ID"])
print("Total subjects with effects: %i" %len(subjects_with_effects))
print("Total diagnoses with effects: %i" %len(diagnoses_with_effects_df))

Total subjects with effects: 905
Total diagnoses with effects: 1338


In [28]:
diagnoses_with_effects_df

ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM ICD9_CODE
277        506          64   172056      4.0     30400
396        625          78   100536      1.0     96500
400        629          78   100536      5.0     30401
403        632          78   100536      8.0     E8502
1340      1229         109   158995      1.0     96509
1351      1240         109   158995     12.0     E8502
1408      3270         273   158689     12.0     E8500
2035      3571         303   103013      6.0     96509
2295      1799         143   143808      6.0     30401
2454      2121         172   148505      4.0     30401
2468      2135         174   110104      3.0     30551
2815       853         106   145167      6.0     30550
3592      3259         273   158689      1.0     96501
3593      3260         273   158689      2.0     30421
3597      3264         273   158689      6.0     30401
3600      3267         273   158689      9.0     30431
3727      5593         468   109718      1.0     96500
4872      3834         326   188926      1.0     96501
4874      3836         326   188926      3.0     30471
4875      3837         326   188926      4.0     E8500
4877      3839         326   188926      6.0      9701
5846      5807         493   165956      2.0     30540
5895      5856         497   152166      9.0     30551
6612      7877         689   149102     15.0     96502
6615      7880         689   149102     18.0     E8501
8071      8303         720   171046      9.0     30550
8206      9906         852   162589      2.0     30401
8249      9949         855   173950      2.0     30401
11091    10518         902   137798      2.0     30441
11097    10524         902   137798      8.0     30431
...        ...         ...      ...      ...       ...
636059  648506       99444   136391      8.0     30541
637170  640586       97652   187837     27.0     30550
637341  634909       96520   145731      1.0      9701
637488  635056       96553   194204     14.0     30400
638540  611469       91373   116853      5.0     30410
638541  611470       91373   116853      6.0     30400
640331  635607       96669   193592      8.0     30401
640469  635745       96686   138595     20.0     30550
640516  635792       96686   158777     18.0     30550
641539  636051       96705   107813      2.0     30401
641540  636052       96705   107813      3.0     30421
641818  643028       98229   149609      9.0     30550
642419  636166       96731   198001      7.0     30431
644253  643721       98389   155368      7.0     30401
645360  630694       95574   125789      8.0     30421
645717  632767       96049   166616      1.0     96501
645730  632780       96049   166616     14.0     30550
645737  632787       96049   166616     21.0     E8500
645994  630860       95621   142363     19.0     30400
646110  630976       95639   198041      3.0     30401
646129  630995       95641   139447     13.0     30400
647702  634086       96324   136039      1.0     96501
647968  634098       96324   136039     13.0     30401
648429  640307       97578   191759      1.0     96502
648434  640312       97578   191759      6.0     30401
648618  650319       99830   176834     30.0     30550
649782  637620       96977   109134     18.0     96500
649783  637621       96977   109134     19.0     30551
649784  637622       96977   109134     20.0     E8502
649861  637699       97004   127867     16.0     30550

[1338 rows x 5 columns]

In [29]:
with open('../Data/Opioids/Diagnoses/diagnoses_with_effects_df.pickle', 'wb') as pickler:
    pickle.dump(diagnoses_with_effects_df, pickler, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../Data/Opioids/Diagnoses/subjects_with_effects.pickle', 'wb') as pickler:
    pickle.dump(subjects_with_effects, pickler, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
subjects_311 = set(diagnoses_icd[diagnoses_icd["ICD9_CODE"] == str("311")]["SUBJECT_ID"])
print(len(subjects_311))
drugs_prescribed_311 = dict()
subjects_with_morphine_depression_311 = dict()
for subject in subjects_311:
    drugs_prescribed_311[subject] = set(prescriptions[prescriptions["SUBJECT_ID"] == subject]["DRUG"])
    if ("Morphine" in drugs_prescribed_311[subject]):
        subjects_with_morphine_depression_311[subject] = drugs_prescribed_311[subject]
drugs_prescribed_311

2926


KeyboardInterrupt: 

In [8]:
subjects_296 = set(diagnoses_icd[diagnoses_icd["ICD9_CODE"] == str("29690")]["SUBJECT_ID"])
print(len(subjects_296))
drugs_prescribed_296 = dict()
subjects_with_morphine_depression_296 = dict()
for subject in subjects_296:
    drugs_prescribed_296[subject] = set(prescriptions[prescriptions["SUBJECT_ID"] == subject]["DRUG"])
    if ("Morphine" in drugs_prescribed_296[subject]):
        subjects_with_morphine_depression_296[subject] = drugs_prescribed_296[subject]
print(len(subjects_with_morphine_depression_296))

69
0


In [9]:
subjects_29699 = set(diagnoses_icd[diagnoses_icd["ICD9_CODE"] == str("29699")]["SUBJECT_ID"])
print(len(subjects_29699))
drugs_prescribed_29699 = dict()
subjects_with_morphine_depression_29699 = dict()
for subject in subjects_29699:
    drugs_prescribed_29699[subject] = set(prescriptions[prescriptions["SUBJECT_ID"] == subject]["DRUG"])
    if ("Morphine" in drugs_prescribed_29699[subject]):
        subjects_with_morphine_depression_29699[subject] = drugs_prescribed_29699[subject]
print(len(subjects_with_morphine_depression_29699))

4
0


In [27]:
subjects_309 = diagnoses_icd[diagnoses_icd["ICD9_CODE"] == str("3110")]
print(len(subjects_3000))
''''drugs_prescribed_3000 = dict()
subjects_with_morphine_depression_29699 = dict()
for subject in subjects_29699:
    drugs_prescribed_29699[subject] = set(prescriptions[prescriptions["SUBJECT_ID"] == subject]["DRUG"])
    if ("Morphine" in drugs_prescribed_29699[subject]):
        subjects_with_morphine_depression_29699[subject] = drugs_prescribed_29699[subject]
print(len(subjects_with_morphine_depression_29699))'''

0


'\'drugs_prescribed_3000 = dict()\nsubjects_with_morphine_depression_29699 = dict()\nfor subject in subjects_29699:\n    drugs_prescribed_29699[subject] = set(prescriptions[prescriptions["SUBJECT_ID"] == subject]["DRUG"])\n    if ("Morphine" in drugs_prescribed_29699[subject]):\n        subjects_with_morphine_depression_29699[subject] = drugs_prescribed_29699[subject]\nprint(len(subjects_with_morphine_depression_29699))'